## Check the gamma moments

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from dataset import Dataset

In [2]:
μ_0 = np.array([0.75,0.25])
cov_0 = np.array([[0.01, 0.005], [0.005, 0.01]])

μ_1 = np.array([0.25, 0.75])
cov_1 = np.array([[0.01, 0.005], [0.005, 0.01]])

gamma_prior_cov = np.array([0.001]) #0.01*np.array([0.1])

ν = cov_0[1,1]


# Number of samples to generate
n_samples = 1000

# Generate samples alternately
samples = np.zeros((n_samples, 2))
for i in range(n_samples):
    if i % 2 == 0:
        samples[i] = np.random.multivariate_normal(μ_0, cov_0)
    else:
        samples[i] = np.random.multivariate_normal(μ_1, cov_1)

#print(f"==>> samples: {samples}")


ds = Dataset(samples, emb_dim=2, N=n_samples, K=2)

for i in range(0,len(ds.z_vars)):
    ds.z_vars[i].probs = [1.0, 0.0] if i % 2 == 0 else [0.0, 1.0]

assumed_dof = 5 #= d+3

ds.means_vars[0].prior_cov = cov_0
ds.means_vars[0].mean = μ_0
ds.means_vars[0].cov = cov_0

ds.means_vars[1].prior_cov = cov_1
ds.means_vars[1].mean = μ_1
ds.means_vars[1].cov = cov_1


ds.gamma_vars[0].prior_cov = np.array([gamma_prior_cov / ν])
ds.gamma_vars[0].mean = np.array([cov_0[0, 1]]) / np.sqrt(ν)
ds.gamma_vars[0].cov = np.array([gamma_prior_cov / ν])
ds.gamma_vars[0].nu = cov_0[-1,-1]

ds.gamma_vars[1].prior_cov = np.array([gamma_prior_cov / ν])
ds.gamma_vars[1].mean = np.array([cov_1[0, 1]]) / np.sqrt(ν)
ds.gamma_vars[1].cov = np.array([gamma_prior_cov / ν])
ds.gamma_vars[1].nu = cov_1[-1,-1]

ds.sigma_star_vars[0].prior_scale = np.array([cov_0[0,0] - ds.gamma_vars[0].mean ** 2]) * (assumed_dof - ds.d)  # np.array([cov_0[0,0] * (assumed_dof - ds.d)])
ds.sigma_star_vars[0].dof = 5
ds.sigma_star_vars[0].prior_dof = 5
ds.sigma_star_vars[0].scale = np.array([cov_0[0,0] - ds.gamma_vars[0].mean ** 2]) * (assumed_dof - ds.d)
ds.sigma_star_vars[0].nu = cov_1[-1,-1]           

ds.sigma_star_vars[0].first_moment = ds.sigma_star_vars[0].first_mom()
ds.sigma_star_vars[0].second_moment = ds.sigma_star_vars[0].second_mom()     


ds.sigma_star_vars[1].prior_scale = np.array([cov_1[0,0] - ds.gamma_vars[1].mean ** 2]) * (assumed_dof - ds.d)  #np.array([[cov_1[0,0] * (assumed_dof - ds.d)]])
ds.sigma_star_vars[1].dof = 5
ds.sigma_star_vars[1].prior_dof = 5
ds.sigma_star_vars[1].scale = np.array([cov_1[0,0] - ds.gamma_vars[1].mean ** 2]) * (assumed_dof - ds.d)
ds.sigma_star_vars[1].nu = cov_1[-1,-1]

ds.sigma_star_vars[1].first_moment = ds.sigma_star_vars[1].first_mom()
ds.sigma_star_vars[1].second_moment = ds.sigma_star_vars[1].second_mom()

full_sigma_inv_estimates = [np.linalg.inv(cov_mat) for cov_mat in [cov_0, cov_1]]



for i, (r_var, z_var) in enumerate(zip(ds.r_vars, ds.z_vars)):
    C=0
    D=0
    norm_datapoint = ds.normed_embds[i]
    norm_datapoint = norm_datapoint.reshape(-1, 1)

    for k in range (0, len(ds.z_vars[i].probs)):
        data_group = k
        sigma = ds.sigma_star_vars[data_group]
        γ = ds.gamma_vars[data_group]
        μ = ds.means_vars[data_group]

        sigma_inv = full_sigma_inv_estimates[data_group]

        C += z_var.probs[k] * np.matmul(np.matmul(norm_datapoint.T, sigma_inv), norm_datapoint)
        D += z_var.probs[k] * np.matmul(np.matmul(norm_datapoint.T, sigma_inv), μ.mean)

    C = np.reshape(C, -1)
    D = np.reshape(D, -1)

    r_var.alpha = C / 2
    r_var.beta = D / C

    r_var.update_moments(norm_datapoint)

    # r_var.first_moment = np.linalg.norm(ds.embds[i])

    
    # initialise phi variables

# ds.phi_var.conc[0] = n_samples // 2
# ds.phi_var.conc[1] = n_samples // 2 


ds.dataset_vi(max_iter=5, real_cov=cov_0)

Iteration 0 results:
                  
                    μ_0_mean: [0.75 0.25]
                    μ_0_cov: [[0.01  0.005]
 [0.005 0.01 ]]

                    μ_1_mean: [0.25 0.75]
                    μ_1_cov: [[0.01  0.005]
 [0.005 0.01 ]]


                 _____________________________________________________________________

                    sigma_0_scale: [[0.0225]]
                    sigma_0_prior_scale: [[0.0225]]
                    sigma_0_dof: 5
                    sigma_0_prior_dof: 5
                    sigma_0_first_moment: [[0.0075]]
                    sigma_0_mode: [[0.00321429]]
                    real_sigma_star: [0.0075]


                    sigma_1_scale: [[0.0225]]
                    sigma_1_prior_scale: [[0.0225]]
                    sigma_1_dof: 5
                    sigma_1_prior_dof: 5
                    sigma_1_first_moment: [[0.0075]]
                    sigma_1_mode: [[0.00321429]]
                    real_sigma_star: [0.0075]




               

In [4]:
gamma_var = ds.gamma_vars[0]

print(gamma_var.mean)
print(gamma_var.cov)




[[[0.04991064]]]
[[[9.1058046e-06]]]


In [6]:
print(gamma_var.outer_prod())

[[[0.00250018]]]


In [7]:
print(gamma_var.outer_product)

[[[0.00250018]]]


In [8]:
print(gamma_var.corr)

[[[1.]]]


In [11]:
print(gamma_var.three_gamma())

[[0.00012569]]


In [18]:
from gamma_dist import Gamma

new_gamma = Gamma(0, 2)
new_gamma.mean = np.array([0.5])
new_gamma.cov = np.array([[0.75]])

print(new_gamma.outer_prod())

print(new_gamma.three_gamma())

print(new_gamma.quadruple_gamma())

[[1.]]
[[1.25]]
[[2.875]]


In [3]:
import numpy as np
import matplotlib.pyplot as plt

def generate_random_matrix_and_covariance():
    # Generate a random 2x2 matrix
    A = np.random.rand(2, 2)
    
    # Compute the covariance matrix (A @ A.T ensures it's positive semi-definite)
    cov_matrix = A @ A.T
    
    return A, cov_matrix

def generate_random_mean():
    # Generate a random mean vector of size 2
    mean = np.random.rand(2)
    
    return mean

def monte_carlo_approximation_three(mean, cov_matrix, num_samples=10000):
    # Generate samples from a multivariate normal distribution
    samples = np.random.multivariate_normal(mean, cov_matrix, num_samples)
    
    # Monte Carlo simulation to approximate E[γγ^Tγ]
    results = []
    for gamma in samples:
        result = np.dot(np.dot(gamma.T, gamma), gamma)
        results.append(result)
    
    # Taking the mean of the results as the Monte Carlo approximation
    approx = np.mean(results, axis=0)
    
    return approx

def monte_carlo_approximation_quad(mean, cov_matrix, num_samples=10):
    # Generate samples from a multivariate normal distribution
    samples = np.random.multivariate_normal(mean, cov_matrix, num_samples)
    
    # Monte Carlo simulation to approximate E[γγ^T γγ^T]
    results = []
    for gamma in samples:
        gamma_gammaT = np.outer(gamma, gamma.T)  # Compute γγ^T
        result = np.dot(gamma_gammaT, gamma_gammaT)  # Compute γγ^T γγ^T
        # print(f"==>> result_1 : {result}")

        # print(f"==>>result 2: {np.outer(gamma, gamma) @ np.outer(gamma, gamma)}")

        results.append(result)
    
    # Taking the mean of the results as the Monte Carlo approximation
    approx = np.mean(results, axis=0)
    
    return approx

# Main execution
A, cov_matrix = generate_random_matrix_and_covariance()
mean = generate_random_mean()
approx = monte_carlo_approximation_quad(mean, cov_matrix)

print("Random 2x2 Matrix A:")
print(A)
print("\nCovariance Matrix derived from A:")
print(cov_matrix)
print("\nRandom Mean Vector:")
print(mean)
print("\nMonte Carlo approximation of E[γγ^Tγ]:")
print(approx)


Random 2x2 Matrix A:
[[0.22839738 0.00361721]
 [0.88078595 0.85625916]]

Covariance Matrix derived from A:
[[0.05217845 0.20426647]
 [0.20426647 1.50896364]]

Random Mean Vector:
[0.79799681 0.90698891]

Monte Carlo approximation of E[γγ^Tγ]:
[[ 3.60077426  7.95377527]
 [ 7.95377527 20.7418933 ]]


In [33]:
cov_matrix = np.array([[1.01884864, 1.18926507],
 [1.18926507, 1.42588716]])

mean = np.array([0.5, 0.7])
monte_carlo_approximation(mean, cov_matrix, num_samples=100_000)

array([4.28548174, 5.42482877])

In [28]:
new_gamma = Gamma(0, 3)
new_gamma.mean = mean
new_gamma.cov = cov_matrix

print(new_gamma.three_gamma())

[[4.27618764]
 [5.41482215]]


In [48]:
for _ in range(10):
    A, cov_mat = generate_random_matrix_and_covariance()
    mean = generate_random_mean()
    approx = monte_carlo_approximation(mean, cov_matrix)
    new_gamma = Gamma(0, 3)
    new_gamma.mean = mean
    new_gamma.cov = cov_matrix

    print(f"approx: {approx}")
    print(f"three_gamma: {new_gamma.three_gamma()}")
    print()
    

approx: [2.7907245  3.64073102]
three_gamma: [[2.773786  ]
 [3.61814632]]

approx: [3.7327041 5.1550795]
three_gamma: [[3.79878518]
 [5.24367549]]

approx: [3.34911844 2.27626489]
three_gamma: [[3.33662096]
 [2.26418693]]

approx: [2.32831174 1.19953423]
three_gamma: [[2.363893  ]
 [1.23079506]]

approx: [1.12398851 0.84903434]
three_gamma: [[1.17956078]
 [0.91231723]]

approx: [ 9.74299589 10.11051568]
three_gamma: [[ 9.92899714]
 [10.32061222]]

approx: [6.72723544 4.11640929]
three_gamma: [[6.71889859]
 [4.11558948]]

approx: [8.31834105 7.23315134]
three_gamma: [[8.37123527]
 [7.2802257 ]]

approx: [5.93664405 3.62550628]
three_gamma: [[6.02554484]
 [3.69418388]]

approx: [3.4952437  4.57554069]
three_gamma: [[3.46227964]
 [4.53259129]]



In [7]:
import numpy as np

def exact_expectation(mean, cov_matrix):
    mu1, mu2 = mean[0], mean[1]
    sigma11 = cov_matrix[0, 0]
    sigma22 = cov_matrix[1, 1]
    rho = cov_matrix[0, 1] / np.sqrt(sigma11 * sigma22)

    # Compute the required expectations
    E_gamma1_4 = 3 * sigma11**2 + 6 * sigma11 * mu1**2 + mu1**4
    E_gamma2_4 = 3 * sigma22**2 + 6 * sigma22 * mu2**2 + mu2**4
    E_gamma1_2_gamma2_2 = sigma11 * sigma22 + sigma11 * mu2**2 + sigma22 * mu1**2 + mu1**2 * mu2**2

    
    # E_gamma1_3_gamma2 = sigma11 * 3 * mu1 * mu2 + mu1**3 * mu2
    # E_gamma1_gamma2_3 = sigma22 * 3 * mu1 * mu2 + mu1 * mu2**3

    E_gamma1_3_gamma2 = (mu1**3 + 3*mu1*sigma11) * (mu2 - ((rho * np.sqrt(sigma11) * mu1) / np.sqrt(sigma22))) + ((rho * np.sqrt(sigma11) / np.sqrt(sigma22)) * (mu1**4 + 6*mu1**2*sigma11 + 3*sigma11**2))
    E_gamma1_gamma2_3 = (mu2**3 + 3*mu2*sigma22) * (mu1 - ((rho * np.sqrt(sigma22) * mu2) / np.sqrt(sigma11))) + ((rho * np.sqrt(sigma22) / np.sqrt(sigma11)) * (mu2**4 + 6*mu2**2*sigma22 + 3*sigma22**2))

    # Construct the expectation matrix
    expectation_matrix = np.array([
        [E_gamma1_4 + E_gamma1_2_gamma2_2, E_gamma1_3_gamma2 + E_gamma1_gamma2_3],
        [E_gamma1_3_gamma2 + E_gamma1_gamma2_3, E_gamma2_4 + E_gamma1_2_gamma2_2]
    ])

    return expectation_matrix

# Main execution
A, cov_matrix = generate_random_matrix_and_covariance()
mean = generate_random_mean()
exact_matrix = exact_expectation(mean, cov_matrix)

print("Random 2x2 Matrix A:")
print(A)
print("\nCovariance Matrix derived from A:")
print(cov_matrix)
print("\nRandom Mean Vector:")
print(mean)
print("\nExact expectation of the matrix expression:")
print(exact_matrix)


Random 2x2 Matrix A:
[[0.69017954 0.22430623]
 [0.0961748  0.14923871]]

Covariance Matrix derived from A:
[[0.52666109 0.09985305]
 [0.09985305 0.03152178]]

Random Mean Vector:
[0.83374349 0.32397389]

Exact expectation of the matrix expression:
[[3.6786516  6.78591424]
 [6.78591424 0.20059912]]


In [8]:


import numpy as np

# def generate_random_matrix_and_covariance():
#     diag_elements = np.random.rand(2)
#     cov_matrix = np.diag(diag_elements)
#     return diag_elements, cov_matrix

def generate_random_matrix_and_covariance():
    # Generate a random 2x2 matrix
    A = np.random.rand(2, 2)
    
    # Compute the covariance matrix (A @ A.T ensures it's positive semi-definite)
    cov_matrix = A @ A.T
    
    return A, cov_matrix

def generate_random_mean():
    # Generate a random mean vector of size 2
    mean = np.random.rand(2)
    
    return mean



In [10]:
from gamma_dist import Gamma

for _ in range(5):
    A, cov_mat = generate_random_matrix_and_covariance()


    mean = generate_random_mean()

    # print(f"mean: {mean}")
    # print(f"around the mean: {np.outer(mean, mean) @ np.outer(mean, mean)}")

    approx = monte_carlo_approximation_quad(mean, cov_mat, num_samples=1_000_000)

    new_gamma = Gamma(0, 3)
    new_gamma.mean = mean
    new_gamma.cov = cov_mat
    
    print(f"approx: {approx}")
    exact_ex = exact_expectation(mean, cov_mat)

    print(f"exact: {exact_ex}")
    print(f"quad_gamma: {new_gamma.quadruple_gamma()}")
    print()
    

approx: [[0.19956041 0.42532216]
 [0.42532216 1.28024846]]
exact: [[0.12671044 1.38144961]
 [1.38144961 1.20579638]]
quad_gamma: [[0.1976561  0.42438833]
 [0.42438833 1.26833456]]

approx: [[ 5.67091168  9.09495295]
 [ 9.09495295 14.76461718]]
exact: [[ 3.15779147 12.42913393]
 [12.42913393 12.26424553]]
quad_gamma: [[ 4.98628703  9.10871157]
 [ 9.10871157 14.54066217]]

approx: [[4.7044828  2.83747189]
 [2.83747189 2.17550335]]
exact: [[4.26854352 8.22963125]
 [8.22963125 1.72974813]]
quad_gamma: [[9.22027641 2.84407272]
 [2.84407272 3.3623133 ]]

approx: [[6.45081052 3.90641138]
 [3.90641138 2.89882633]]
exact: [[5.51069791 4.4847987 ]
 [4.4847987  1.9583119 ]]
quad_gamma: [[6.39230472 3.89972863]
 [3.89972863 2.73499353]]

approx: [[11.34964406 14.05572239]
 [14.05572239 17.97393173]]
exact: [[ 7.08390084 15.60294956]
 [15.60294956 13.71832068]]
quad_gamma: [[11.24563677 14.08332255]
 [14.08332255 17.89400867]]

